<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-12 14:25:41
-------------------
qualified stocks: 96
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  1.58 L
Current:  1.26 C
-------------------
Today PnL: 4.36 L (3.59%)
Current PnL: -19.67 L (-13.95%)
CY Booked + Current PnL: -16.84 L (-11.94%)
-------------------
Total profit:  3.69 L
Total loss:  -23.36 L
-------------------
Total Booked + Current PnL: 10.14 L (8.89%)
Total Booked PnL: 29.81 L (26.14%)
Curr Year Booked PnL: 2.83 L (2.25%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 82.37 L (65.4%)
-------------------
Est LTT:  2.70 C
Est LTT PnL: 1.44 C (114.44%)
Deployed:  1.14 C
Current:  1.26 C
CAGR/XIRR %: 7.2%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
53         MRF  114777.60  150504.14    21.99       X-MC  89.58  140135.0   
71     SBILIFE    1496.49    1924.99    -0.88       H-LC  84.38  147210.0   
58  PIDILITIND    2504.06    3576.00   -11.59       X-LC  50.00   92262.0   
72        SBIN     760.30     863.00   -15.73       M-LC  61.46  212610.0   
56        NMDC      65.62      82.00    34.50       M-MC  59.38  104086.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53      25357.0  10370.0        1.79         22.09   7.40  31.13    215.0   
71      21505.0  14485.0        3.10         17.11   9.84  28.63    116.0   
58      17140.0  15020.0        3.20         22.82  16.28  42.81     87.0   
72      11130.0  16095.0        2.96          5.52   7.57  13.51     91.0   
56       3687.0  21379.0        5.82          3.67  20.54  24.96     73.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
53     2.45        1.13    36.50       AR      ATH      TYRES  
71     1.48        1.18    31.14       AR      ATH  INSURANCE  
58     1.14        0.74    15.71      X40      BTT  CHEMICALS  
72     0.69        1.71    16.48     XY25      NTT      BANKS  
56     0.17        0.84    17.31     XY25      NTT     MINING

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost       FTT  Dev%_PE Conviction    RSP   Current  \
19        CERA  8421.60  10420.45   112.85       X-SC  94.79   85066.0   
90  VALIANTORG   512.64    838.00  -294.01       H-SC   1.04   82446.0   
67        SAIL   130.64    228.00    78.75       M-MC  58.33  203420.0   
84    TITAGARH  1097.23   1548.00   -25.94       H-SC   7.29  129946.0   
60       QUESS   711.66    986.00      NaN        NaN  97.92   77679.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
19     -24415.0   50402.0       12.20        -22.30   59.25  23.73    122.0   
90     -50840.0  135434.0        9.99        -38.14  164.27  63.47     83.0   
67     -21542.0  189201.0        8.37         -9.58   93.01  74.53    217.0   
84     -62069.0  140952.0        8.05        -32.33  108.47  41.08     76.0   
60     -78175.0  138253.0        7.75        -50.16  177.98  38.55      NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy     Category  
19    -0.48        0.68    28.75     X40N      ATH     CERAMICS  
90    -0.38        0.66    38.20       XR      NTT    CHEMICALS  
67    -0.11        1.64    18.11     XY24      BTT        STEEL  
84    -0.44        1.04     9.98     XY24      NTT  ENGINEERING  
60    -0.57        0.62    18.95     XY24      NTT         MISC

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
39  INDUSINDBK  1354.50  1800.00   -39.16       H-MC  72.92   44930.0   
16       BSOFT   628.85   844.81   -28.35       M-SC  85.42   67279.0   
40        INFY  1432.81  2275.00   -16.01       X-LC  88.54  209556.0   
12   BANKINDIA   113.49   190.00   -28.40       M-MC  27.08  125214.0   
69       SATIN   199.77   274.00   -17.21       H-SC  80.21  164720.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
39     -32276.0   57668.0       -3.66        -41.81  128.35  32.89     41.0   
16     -42770.0   80567.0       -0.39        -38.86  119.75  34.34     20.0   
40      10395.0  106664.0       -0.25          5.22   50.90  58.78      9.0   
12      -3938.0   91006.0       -0.17         -3.05   72.68  67.42     71.0   
69     -31254.0  104070.0       -0.17        -15.95   63.18  37.16    142.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
39    -0.56        0.36    23.73       XR      NTT    BANKS  
16    -0.53        0.54    10.76       XR      ATH       IT  
40     0.10        1.68     9.41      X40      BTT       IT  
12    -0.04        1.01    21.78       XR      NTT    BANKS  
69    -0.30        1.32    23.01     XY24      NTT  FINANCE

In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
82  TATAMOTORS   776.02  1065.00   -52.64       M-LC  95.83  190999.0   
2          ABB  5319.37  8953.12   -38.31       H-LC  76.04  139900.0   
24         DLF   683.35  1031.70   -26.21       H-LC  46.88  122337.0   
10         AWL   342.88   485.00   -69.33       H-MC   6.25  166243.0   
94   WHIRLPOOL  1151.42  2270.00   -36.05       M-SC  92.71  161163.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
82     -14646.0   91221.0        1.73         -7.12  47.76  37.24      1.0   
2        6916.0   83926.0        2.86          5.20  59.99  68.31      2.0   
24       -666.0   63371.0        7.59         -0.54  51.80  50.98      3.0   
10     -51486.0  141739.0        3.64        -23.65  85.26  41.45      5.0   
94      14933.0  127125.0        3.77         10.21  78.88  97.15      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
82    -0.16        1.54    24.32     XY24      NTT        AUTO  
2      0.08        1.12    14.08       AR      ATH  ELECTRICAL  
24    -0.01        0.98    11.78       AR      ATH      REALTY  
10    -0.36        1.34    11.05     XY24      NTT        FMCG  
94     0.12        1.30    37.68      X40      NTT    DURABLES

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
53         MRF  114777.60  150504.14    21.99       X-MC  89.58  140135.0   
52     MEDANTA    1022.60    1486.00     5.95       X-MC  13.54  186793.0   
58  PIDILITIND    2504.06    3576.00   -11.59       X-LC  50.00   92262.0   
4   ADANIPORTS    1103.69    1583.00     5.78       M-LC  86.46  186868.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53      25357.0  10370.0        1.79         22.09   7.40  31.13    215.0   
52      31358.0  39077.0        5.00         20.17  20.92  45.32    198.0   
58      17140.0  15020.0        3.20         22.82  16.28  42.81     87.0   
4       35662.0  30011.0        4.42         23.59  16.06  43.43     37.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
53     2.45        1.13    36.50       AR      ATH       TYRES  
52     0.80        1.50    27.10     XY24      NTT  HEALTHCARE  
58     1.14        0.74    15.71      X40      BTT   CHEMICALS  
4      1.19        1.50    29.53     XY24      NTT        MISC

In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
55        NHPC    82.08   115.00   193.78       M-MC  18.75  142560.0   
56        NMDC    65.62    82.00    34.50       M-MC  59.38  104086.0   
59     POLYCAB  5005.92  7541.35    18.00       H-MC  90.62  167664.0   
4   ADANIPORTS  1103.69  1583.00     5.78       M-LC  86.46  186868.0   
6     ANGELONE  2328.67  3033.00    -1.24       H-SC  71.88  161200.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
55       1547.0  55014.0        6.43          1.10  38.59  40.11    212.0   
56       3687.0  21379.0        5.82          3.67  20.54  24.96     73.0   
59      27498.0  43492.0        3.84         19.62  25.94  50.65     54.0   
4       35662.0  30011.0        4.42         23.59  16.06  43.43     37.0   
6        9836.0  35948.0        6.60          6.50  22.30  30.25    131.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
55     0.03        1.15    16.54     XY24      NTT    POWER  
56     0.17        0.84    17.31     XY25      NTT   MINING  
59     0.63        1.35    27.96     X40N      ATH   CABLES  
4      1.19        1.50    29.53     XY24      NTT     MISC  
6      0.27        1.30    27.59     X40N      NTT  FINANCE

In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
95      WIPRO   243.46   420.00    -6.99       M-LC  63.54  159619.0   
18       CAMS  3643.00  5250.99    -7.44       H-SC  10.42  102404.0   
37  INDIAMART  2311.97  4911.36   -53.33       H-SC  75.00  100848.0   
93     VOLTAS  1272.00  1929.20   -18.15       H-MC  22.92  139282.0   
12  BANKINDIA   113.49   190.00   -28.40       M-MC  27.08  125214.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
95       8674.0  100783.0        6.38          5.75   63.14   72.51     58.0   
18        400.0   44628.0        5.11          0.39   43.58   44.14     47.0   
37       -879.0  115249.0        1.61         -0.86  114.28  112.43      NaN   
93       -638.0   72928.0        2.48         -0.46   52.36   51.67     19.0   
12      -3938.0   91006.0       -0.17         -3.05   72.68   67.42     71.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
95     0.09        1.28    20.18       XR      NTT       IT  
18     0.01        0.82    18.62       XR      ATH     MISC  
37    -0.01        0.81    19.02       AR      ATH     MISC  
93    -0.01        1.12     5.20       AR      ATH       AC  
12    -0.04        1.01    21.78       XR      NTT    BANKS

In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
87  UJJIVANSFB      52.77      60.00   -40.07       M-SC  78.12  115965.0   
53         MRF  114777.60  150504.14    21.99       X-MC  89.58  140135.0   
88   UNIONBANK     123.87     163.00    -8.12       M-LC  57.29  145070.0   
71     SBILIFE    1496.49    1924.99    -0.88       H-LC  84.38  147210.0   
52     MEDANTA    1022.60    1486.00     5.95       X-MC  13.54  186793.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
87     -26514.0  46038.0        5.45        -18.61  39.70  13.70    219.0   
53      25357.0  10370.0        1.79         22.09   7.40  31.13    215.0   
88       4230.0  40257.0        3.82          3.00  27.75  31.59     83.0   
71      21505.0  14485.0        3.10         17.11   9.84  28.63    116.0   
52      31358.0  39077.0        5.00         20.17  20.92  45.32    198.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
87    -0.58        0.93    36.87     X40N      NTT       BANKS  
53     2.45        1.13    36.50       AR      ATH       TYRES  
88     0.11        1.17    26.01     XY24      NTT       BANKS  
71     1.48        1.18    31.14       AR      ATH   INSURANCE  
52     0.80        1.50    27.10     XY24      NTT  HEALTHCARE

In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction     RSP   Current  \
92      VIPIND   488.80   718.0  -800.56       H-SC  100.00   66748.0   
81   TATAELXSI  7332.28  9000.0   -19.01       X-MC   98.96   85050.0   
60       QUESS   711.66   986.0      NaN        NaN   97.92   77679.0   
1     AARTIIND   487.04   919.0    23.23       M-SC   96.88   77939.0   
82  TATAMOTORS   776.02  1065.0   -52.64       M-LC   95.83  190999.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
92     -28568.0   73263.0        3.59        -29.97  109.76  46.89     51.0   
81     -17602.0   40952.0        5.87        -17.15   48.15  22.74     33.0   
60     -78175.0  138253.0        7.75        -50.16  177.98  38.55      NaN   
1       -3397.0   75531.0        3.34         -4.18   96.91  88.69    118.0   
82     -14646.0   91221.0        1.73         -7.12   47.76  37.24      1.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
92    -0.39        0.54    31.65     X40N      NTT       MISC  
81    -0.43        0.68    27.84     X40N      NTT         IT  
60    -0.57        0.62    18.95     XY24      NTT       MISC  
1     -0.04        0.63    31.43       XR      NTT  CHEMICALS  
82    -0.16        1.54    24.32     XY24      NTT       AUTO

In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.06
1     25       40.47
2     50       68.40

In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.11
X40     14.23
X40N    13.75
AR      13.55
XY25    11.01
XR      10.04
X200     1.19
SR       1.09
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.64
X-LC    18.06
M-SC    16.12
M-LC     8.33
M-MC     7.38
X-MC     6.69
H-LC     6.67
H-MC     6.65
X-SC     2.76
L-SC     2.21
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                11.69         -2.04   34.50
BANKS               10.85        -11.60   66.34
IT                   8.99        -23.60   92.56
FINANCE              7.19        -23.70   79.52
MISC                 7.03        -29.38   85.06
INSURANCE            5.92         -4.91   36.54
PAINTS               5.53        -19.44   54.78
ELECTRICAL           4.02        -10.98   67.53
REFINERIES           3.64         10.09    6.48
HEALTHCARE           3.59         -4.22   37.88
CHEMICALS            3.19        -18.92  100.85
AUTO                 3.17        -15.33   66.95
FOOTWEAR             2.38        -39.24   85.25
POWER                2.02        -13.06   58.29
DURABLES             1.97         -1.26   79.89
AC                   1.92        -12.17   72.01
TRAVEL               1.91        -33.33   91.03
TEXTILES             1.65        -11.97   70.78
CERAMICS             1.65        -47.89  110.35
STEEL                1.64        -10.59   93.01
CABLES               1.35         16.40   25.94
JEWELLERY            1.22        -90.91  151.28
TYRES                1.13         18.09    7.40
CEMENT               1.06        -42.36  115.61
PHARMA               1.06          0.43   48.51
ENGINEERING          1.04        -47.77  108.47
REALTY               0.98         -0.54   51.80
ENTERTAINMENT        0.88        -49.10  124.07
MINING               0.84          3.54   20.54
HOTELS               0.46        -96.48  165.78

In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3179819.0     28
XR        1294418.0     15
AR        1270640.0     15
X40N      1222454.0     17
X40        822840.0     12
SR         208995.0      2
XY25       168148.0      6
X200        69559.0      1

In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2495779.0     22
M-SC        2000579.0     20
X-LC         733396.0     12
M-MC         674472.0      7
H-MC         401683.0      6
X-SC         365617.0      4
M-LC         344881.0      6
X-MC         322763.0      6
L-SC         322602.0      4
H-LC         263780.0      6

In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      947849.0      6
M-SC       XY24      798782.0      6
H-SC       X40N      570117.0      7
           AR        560576.0      3
M-MC       XY24      476506.0      4
M-SC       XR        442819.0      5
X-LC       X40       382910.0      5
M-SC       X40N      379760.0      5
H-SC       XR        312572.0      4
M-LC       XY24      228003.0      4
M-SC       X40       200446.0      2
H-LC       AR        193522.0      4
X-LC       AR        188213.0      3
H-MC       XY24      185154.0      2
L-SC       XR        164887.0      2
X-MC       XY24      141401.0      2
X-SC       XR        124683.0      1
           SR        122761.0      1
           X40N      118173.0      2
X-MC       X40N      110912.0      2
H-SC       X40       104665.0      2
M-LC       XR        100783.0      1
X-LC       XY25       92714.0      3
M-SC       AR         92538.0      1
M-MC       XR         91006.0      1
L-SC       XY24       90803.0      1
M-SC       SR         86234.0      1
M-MC       AR         85581.0      1
H-MC       AR         72928.0      1
X-LC       X200       69559.0      1
L-SC       AR         66912.0      1
X-MC       X40        60080.0      1
H-MC       XR         57668.0      1
           X40N       43492.0      1
           X40        42441.0      1
H-LC       XY25       37960.0      1
           X40        32298.0      1
M-MC       XY25       21379.0      1
M-LC       XY25       16095.0      1
X-MC       AR         10370.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
